In [9]:
import cv2
import numpy as np


In [10]:
cap = cv2.VideoCapture('test_video.mp4')

In [11]:
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    return blurred


In [12]:
def detect_edges(blurred_frame):
    edges = cv2.Canny(blurred_frame, 50, 150)
    return edges


In [13]:
def region_of_interest(edges):
    height, width = edges.shape
    mask = np.zeros_like(edges)
    
    polygon = np.array([[
        (int(0.1*width), height),
        (int(0.4*width), int(0.6*height)),
        (int(0.6*width), int(0.6*height)),
        (int(0.9*width), height),
    ]], np.int32)
    
    cv2.fillPoly(mask, polygon, 255)
    cropped_edges = cv2.bitwise_and(edges, mask)
    return cropped_edges


In [14]:
def detect_lines(cropped_edges):
    lines = cv2.HoughLinesP(cropped_edges, rho=1, theta=np.pi/180, threshold=50, 
                            minLineLength=50, maxLineGap=150)
    return lines


In [15]:
def draw_lines(frame, lines):
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    return combo_image


In [16]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    blurred_frame = preprocess_frame(frame)
    edges = detect_edges(blurred_frame)
    cropped_edges = region_of_interest(edges)
    lines = detect_lines(cropped_edges)
    final_frame = draw_lines(frame, lines)
    
    cv2.imshow("Lane Detection", final_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
